In [ ]:
import pandas as pd
import rasterio
from shapely.geometry import box

# Load the CSV file
csv_file = 'coordinates.csv'  # Replace with your CSV file path
coordinates_df = pd.read_csv(csv_file)

# Function to crop the image
def crop_image(image_path, x, y, width, height, output_path):
    with rasterio.open(image_path) as src:
        # Calculate the crop window
        left = x - 10
        bottom = y - 130
        right = left + width
        top = bottom + height
        crop_box = box(left, bottom, right, top)
        
        # Transform crop box to image coordinates
        crop_window = rasterio.features.geometry_window(src, [crop_box], precision=3)
        
        # Crop the image
        cropped_image = src.read(window=crop_window)
        
        # Update the metadata to reflect the new shape
        meta = src.meta.copy()
        meta.update({
            "height": crop_window.height,
            "width": crop_window.width,
            "transform": rasterio.windows.transform(crop_window, src.transform)
        })
        
        # Save the cropped image
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(cropped_image)

# Loop through the coordinates and crop the image
image_path = 'input.tif'  # Replace with your TIF image path
output_dir = 'output'  # Directory to save the cropped images

for index, row in coordinates_df.iterrows():
    twd97_x = row['TWD97_X']
    twd97_y = row['TWD97_Y']
    
    output_path = f"{output_dir}/crop_{index}.tif"
    crop_image(image_path, twd97_x, twd97_y, 40, 120, output_path)
